I'll follow classification process with https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

In [1]:
import time
import numpy as np

import torch
from torch import nn
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

_____

In [2]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)
        
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [3]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True) # embedding + averaging
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

In [4]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

_____

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv('.data/DBpedia/train.csv')
test_df = pd.read_csv('.data/DBpedia/test.csv')
valid_df = pd.read_csv('.data/DBpedia/valid.csv')

In [7]:
label_dict = {v:k for k,v in enumerate(train_df['l3'].unique())}

In [8]:
train_iter = list(zip(train_df['l3'].map(lambda x : label_dict[x]).tolist(), train_df['text'].tolist()))
valid_iter = list(zip(valid_df['l3'].map(lambda x : label_dict[x]).tolist(), valid_df['text'].tolist()))
test_iter = list(zip(test_df['l3'].map(lambda x : label_dict[x]).tolist(), test_df['text'].tolist()))

____

In [9]:
tokenizer = get_tokenizer('basic_english')
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu"); print(device)

# set vocab size
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

cuda:1


In [10]:
# load nn.Module object
num_class = len(set([label for (label, text) in train_iter])); print(num_class)
vocab_size = len(vocab); print(vocab_size)
emsize = 32
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

219
628321


In [11]:
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [12]:
class TemporalDataset(Dataset):
    def __init__(self, mem, transform=None):
        self._data = mem

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        return self._data[idx]

In [13]:
train_dataset = TemporalDataset(train_iter)
valid_dataset = TemporalDataset(valid_iter)
test_dataset = TemporalDataset(test_iter)

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

### generate out of distribution

In [15]:
out_train_data, out_test_data = AG_NEWS()
out_train_dataset = to_map_style_dataset(out_train_data)
out_test_dataset = to_map_style_dataset(out_test_data)
out_train_dataloader = DataLoader(out_train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
out_test_dataloader = DataLoader(out_test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [16]:
label_dict = {1 : 'World',
2 : 'Sports',
3 : 'Business',
4 : "Sci/Tec"}

____

In [17]:
EPOCHS = 10

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

____

# Fine-tuning

In [19]:
from cycler import cycler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [20]:
model.fc = Identity()

In [21]:
class FineTuningModel(nn.Module):

    def __init__(self,
                 n_embed=10000,
                 d_embed=32,
                 d_hidden1=64,
                 d_hidden2=4,
                 d_out=2,
                 dp=0.2,
                 pretrained=None):
        super(FineTuningModel, self).__init__()
        
        self.embed = pretrained
        self.dropout1 = nn.Dropout(dp)
        self.bn1 = nn.BatchNorm1d(d_embed)
        self.fc1 = nn.Linear(d_embed, d_hidden1)
        self.dropout2 = nn.Dropout(dp)
        self.bn2 = nn.BatchNorm1d(d_hidden1)
        self.fc2 = nn.Linear(d_hidden1, d_hidden2)

    def forward(self, text, offsets):
        x = self.embed(text, offsets)

        x = self.dropout1(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.bn2(x)
        x = torch.tanh(self.fc2(x))

        return x

In [22]:
fine_tuning_model = FineTuningModel(pretrained=model).to(device)

In [23]:
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(fine_tuning_model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [24]:
def fine_tune(dataloader):
    fine_tuning_model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        label = label - 1
        optimizer.zero_grad()
        predited_label = fine_tuning_model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(fine_tuning_model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    fine_tuning_model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            label = label - 1
            predited_label = fine_tuning_model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [25]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    fine_tune(out_train_dataloader)
    accu_val = evaluate(out_test_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1875 batches | accuracy    0.650
| epoch   1 |  1000/ 1875 batches | accuracy    0.739
| epoch   1 |  1500/ 1875 batches | accuracy    0.779
-----------------------------------------------------------
| end of epoch   1 | time: 18.60s | valid accuracy    0.835 
-----------------------------------------------------------
| epoch   2 |   500/ 1875 batches | accuracy    0.821
| epoch   2 |  1000/ 1875 batches | accuracy    0.828
| epoch   2 |  1500/ 1875 batches | accuracy    0.839
-----------------------------------------------------------
| end of epoch   2 | time: 18.75s | valid accuracy    0.854 
-----------------------------------------------------------
| epoch   3 |   500/ 1875 batches | accuracy    0.852
| epoch   3 |  1000/ 1875 batches | accuracy    0.856
| epoch   3 |  1500/ 1875 batches | accuracy    0.861
-----------------------------------------------------------
| end of epoch   3 | time: 16.53s | valid accuracy    0.879 
-------------------------------